In [2]:
import os
os.chdir("../")
%pwd

'd:\\End-End DS Project'

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target: str

In [13]:
from src.DataScience.constants import *
from src.DataScience.utils.common import read_yaml, create_directories

## Create the configuration manager class to manage the configuration files
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, 
                       params_filepath = PARAMS_FILE_PATH, 
                       schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)


        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.Elasticnet
        schema = self.schema.TARGET_COLUMN

        # Create the directories for the artifacts
        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target = schema.name
        )
        return model_trainer_config

In [14]:
from sklearn.linear_model import ElasticNet
import os, joblib
import pandas as pd
from src.DataScience import logger

In [15]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train_model(self):
        # Load the training and testing data
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Separate the features and target variable
        X_train = train_data.drop(columns=[self.config.target], axis=1)
        y_train = train_data[self.config.target]
        X_test = test_data.drop(columns=[self.config.target], axis=1)
        y_test = test_data[self.config.target]

        # Initialize the ElasticNet model
        model = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)

        # Train the model
        model.fit(X_train, y_train)

        # Save the model
        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))

In [16]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train_model()
except Exception as e:
    logger.exception(e)
    raise e

[2025-04-16 13:09:12,739: INFO: common: yaml file: config\config.yaml loaded sucessfully]
[2025-04-16 13:09:12,742: INFO: common: yaml file: params.yaml loaded sucessfully]
[2025-04-16 13:09:12,745: INFO: common: yaml file: schema.yaml loaded sucessfully]
[2025-04-16 13:09:12,747: INFO: common: Creating a Directory at: artifacts]
[2025-04-16 13:09:12,747: INFO: common: Creating a Directory at: artifacts/model_trainer]
